In [1]:
!pip uninstall tensorflow
!pip install tensorflow-gpu

In [15]:
import tensorflow as tf
from pathlib import Path
print(tf.__version__)
import os
import numpy as np
from tqdm import tqdm
import pandas as pd
import pdb
from google.colab import drive
drive.mount('/content/drive')

2.2.0
Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [0]:
# data_folder = Path('INSERT YOUR PATH HERE')
data_folder = Path('/content/drive/My Drive/Mary Jane Inc./Data')
player_data = data_folder / 'Player Stats'
team_data = data_folder / 'Team Stats'

In [4]:
years = ['09','10','11','12','13','14','15','16','17','18','19']
for i in tqdm(range(len(years) - 1)):
  if i == 0:
    df_p = pd.read_excel(player_data / f'NBA-20{years[i]}-20{years[i + 1]}-Player-BoxScore-Dataset.xlsx',sheet_name=f'NBA-20{years[i]}-{years[i + 1]}-PLAYER')
    df_t = pd.read_excel(team_data / f'20{years[i]}-20{years[i + 1]}_NBA_Box_Score_Team-Stats.xlsx',sheet_name=f'NBA-20{years[i]}-{years[i + 1]}-TEAM')
  else:
    temp_p = pd.read_excel(player_data / f'NBA-20{years[i]}-20{years[i + 1]}-Player-BoxScore-Dataset.xlsx',sheet_name=f'NBA-20{years[i]}-{years[i + 1]}-PLAYER')
    temp_t = pd.read_excel(team_data / f'20{years[i]}-20{years[i + 1]}_NBA_Box_Score_Team-Stats.xlsx',sheet_name=f'NBA-20{years[i]}-{years[i + 1]}-TEAM')
    df_p = df_p.append(temp_p)
    df_t = df_t.append(temp_t)

100%|██████████| 10/10 [01:11<00:00,  7.13s/it]


In [0]:
def pre_process_player(player_frame):
  player_frame = player_frame.drop(['DATASET', 'PLAYER \nFULL NAME', 'OWN \nTEAM', 'OPPONENT \nTEAM'],axis=1)
  player_frame = player_frame.rename(columns={'VENUE\n(R/H)':'VENUE','STARTER\n(Y/N)':'STARTER','USAGE \nRATE (%)':'USAGE','DAYS\nREST':'REST','GAME-ID':'G_ID','PLAYER-ID':'P_ID'})
  starter_int = {start:int(i) for i, start in enumerate(['Y', 'N'])}
  venue_int = {ven:int(i) for i, ven in enumerate(['H','R'])}
  position_int = {pos:int(i) for i, pos in enumerate(['F', 'C', 'G', 'C-F', 'G-F', 'F-G', 'F-C', 'SF', 'PF', 'PG', 'SG', 'OTHER'])}
  rest_int = {'3+':int(666)}
  player_frame = player_frame.replace({'STARTER':starter_int,'VENUE':venue_int,'POSITION':position_int,'REST':rest_int})
  player_frame['POSITION'] = player_frame['POSITION'].fillna(11.0)
  player_frame['POSITION'] = player_frame['POSITION'].astype('int64')
  player_frame['REST'] = player_frame['REST'].astype('int64')
  player_frame['DATE'] = pd.to_datetime(player_frame['DATE'],format='%m/%d/%Y')
  return player_frame
df_new = df_p
df_new = pre_process_player(df_new)

In [0]:
df_p.head()

In [25]:
num_game_history = 3
for id in df_p['P_ID'].unique():
  dis_person = df_p[df_p['P_ID'] == id]
  for index, row in dis_person.iterrows():
    prev_games = dis_person[dis_person['DATE'] < row['DATE']].sort_values('DATE',ascending=False)
    prev_games.drop(columns=['G_ID','P_ID'])
    if len(prev_games) >= num_game_history:
      prev_games = prev_games.nlargest(num_game_history, 'DATE').values.flatten()


KeyboardInterrupt: ignored

In [0]:
class CVAE(tf.keras.Model):
  """Convolutional variational autoencoder."""

  def __init__(self, latent_dim):
    super(CVAE, self).__init__()
    self.latent_dim = latent_dim
    self.encoder = tf.keras.Sequential(
        [
            tf.keras.layers.InputLayer(input_shape=(28, 28, 1)),
            tf.keras.layers.Conv2D(
                filters=32, kernel_size=3, strides=(2, 2), activation='relu'),
            tf.keras.layers.Conv2D(
                filters=64, kernel_size=3, strides=(2, 2), activation='relu'),
            tf.keras.layers.Flatten(),
            # No activation
            tf.keras.layers.Dense(latent_dim + latent_dim),
        ]
    )

    self.decoder = tf.keras.Sequential(
        [
            tf.keras.layers.InputLayer(input_shape=(latent_dim,)),
            tf.keras.layers.Dense(units=7*7*32, activation=tf.nn.relu),
            tf.keras.layers.Reshape(target_shape=(7, 7, 32)),
            tf.keras.layers.Conv2DTranspose(
                filters=64, kernel_size=3, strides=2, padding='same',
                activation='relu'),
            tf.keras.layers.Conv2DTranspose(
                filters=32, kernel_size=3, strides=2, padding='same',
                activation='relu'),
            # No activation
            tf.keras.layers.Conv2DTranspose(
                filters=1, kernel_size=3, strides=1, padding='same'),
        ]
    )

  @tf.function
  def sample(self, eps=None):
    if eps is None:
      eps = tf.random.normal(shape=(100, self.latent_dim))
    return self.decode(eps, apply_sigmoid=True)

  def encode(self, x):
    mean, logvar = tf.split(self.encoder(x), num_or_size_splits=2, axis=1)
    return mean, logvar

  def reparameterize(self, mean, logvar):
    eps = tf.random.normal(shape=mean.shape)
    return eps * tf.exp(logvar * .5) + mean

  def decode(self, z, apply_sigmoid=False):
    logits = self.decoder(z)
    if apply_sigmoid:
      probs = tf.sigmoid(logits)
      return probs
    return logits

In [0]:
optimizer = tf.keras.optimizers.Adam(1e-4)


def log_normal_pdf(sample, mean, logvar, raxis=1):
  log2pi = tf.math.log(2. * np.pi)
  return tf.reduce_sum(
      -.5 * ((sample - mean) ** 2. * tf.exp(-logvar) + logvar + log2pi),
      axis=raxis)


def compute_loss(model, x):
  mean, logvar = model.encode(x)
  z = model.reparameterize(mean, logvar)
  x_logit = model.decode(z)
  cross_ent = tf.nn.sigmoid_cross_entropy_with_logits(logits=x_logit, labels=x)
  logpx_z = -tf.reduce_sum(cross_ent, axis=[1, 2, 3])
  logpz = log_normal_pdf(z, 0., 0.)
  logqz_x = log_normal_pdf(z, mean, logvar)
  return -tf.reduce_mean(logpx_z + logpz - logqz_x)


@tf.function
def train_step(model, x, optimizer):
  """Executes one training step and returns the loss.

  This function computes the loss and gradients, and uses the latter to
  update the model's parameters.
  """
  with tf.GradientTape() as tape:
    loss = compute_loss(model, x)
  gradients = tape.gradient(loss, model.trainable_variables)
  optimizer.apply_gradients(zip(gradients, model.trainable_variables))

In [0]:
testing = df_p.drop(columns=['DATASET', 'PLAYER \nFULL NAME', 'OWN \nTEAM', 'OPPONENT \nTEAM'])
testing = testing.rename(columns={'VENUE\n(R/H)':'VENUE','STARTER\n(Y/N)':'STARTER','USAGE \nRATE (%)':'USAGE', 'DAYS\nREST':'REST'})
starter_map = {i: start for i, start in enumerate(['Y', 'N'])}
venue_map = {i: ven for i, ven in enumerate(['R', 'H'])}
position_map = {i: pos for i, pos in enumerate(['F', 'C', 'G', 'C-F', 'G-F', 'F-G', 'F-C', 'SF', 'PF', 'PG', 'SG'])}
testing = testing.replace({'STARTER':starter_map, 'VENUE':venue_map,'POSITION':position_map})
testing = testing['POSITION'].fillna(666)
pdb.set_trace()
testing['DATE'] = pd.to_datetime(testing['DATE'],format='%m/%d/%Y')


NameError: ignored